In [ ]:
from google.colab import drive
import pandas as pd
# import cudf
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive")

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/MyDrive")
csv_merged = pd.DataFrame()
df1 = pd.read_csv("/content/drive/MyDrive/ALDA_Project/data/books_preprocessed/Books_5_1.csv")
df2 = pd.read_csv("/content/drive/MyDrive/ALDA_Project/data/books_preprocessed/Books_5_2.csv")
df3 = pd.read_csv("/content/drive/MyDrive/ALDA_Project/data/books_preprocessed/Books_5_3.csv")
df4 = pd.read_csv("/content/drive/MyDrive/ALDA_Project/data/books_preprocessed/Books_5_4.csv")
df5 = pd.read_csv("/content/drive/MyDrive/ALDA_Project/data/books_preprocessed/Books_5_5.csv")

Below we load data from the CSVs stored in the drive and append the csv's together to get more data in the dataframe

Import required libraries like surprise library for SVD and the reader to parse the pandas dataframe

In [ ]:
%pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3195834 sha256=d3056282307a61be9ade06fb69e4d4cabc8e6a845bd79b2bdc87118e7b9d6d7b
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


Import all libaries required for SVD

In [ ]:
csv_merged = csv_merged.append([df1, df2, df3, df4, df5], ignore_index=True)
print(csv_merged)

<ipython-input-4-a9383a3f06ac>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  csv_merged = csv_merged.append([df1, df2, df3, df4, df5], ignore_index=True)


         Unnamed: 0  overall      reviewerID        asin  sentiment
0                 0      5.0  A1REUF3A1YCPHM  0001713353     4.9396
1                 1      5.0   AVP0HXC9FG790  0001713353     4.2720
2                 2      5.0  A324TTUBKTN73A  0001713353     4.7106
3                 3      5.0  A2RE7WG349NV5D  0001713353     4.2738
4                 4      5.0  A32B7QIUDQCD0E  0001713353     4.3176
...             ...      ...             ...         ...        ...
4999371     4999996      4.0  A1K4S4MWXI9E9M  0356500586     4.8592
4999372     4999997      5.0  A30Y5Q30008Y58  0356500586     4.7758
4999373     4999998      3.0   ARTTWKTEF4ERG  0356500586     4.8112
4999374     4999999      3.0   AVRPBDDUT6RW2  0356500586     4.9854
4999375     5000000      3.0  A29L3RMRY5GDPO  0356500586     4.9730

[4999376 rows x 5 columns]


In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise import SVD
import numpy as np
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate

In [ ]:
reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(csv_merged[["asin", "reviewerID", "sentiment"]], reader)

In [ ]:
svd = SVD(n_epochs=10)
results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9885  0.9881  0.9859  0.9861  0.9880  0.9867  0.9883  0.9862  0.9869  0.9878  0.9873  0.0010  
MAE (testset)     0.7428  0.7431  0.7413  0.7409  0.7428  0.7418  0.7429  0.7414  0.7420  0.7420  0.7421  0.0007  
Fit time          95.97   102.87  111.16  109.50  112.67  112.79  114.53  111.36  109.78  110.20  109.08  5.27    
Test time         13.62   13.82   13.68   6.41    6.30    6.37    14.67   13.61   7.42    7.23    10.31   3.59    


In [ ]:
print("Average MAE: ", np.average(results["test_mae"]))
print("Average RMSE: ", np.average(results["test_rmse"]))

Average MAE:  0.7421157042432294
Average RMSE:  0.9872620774382703


In [ ]:
p = csv_merged.groupby('overall')['overall'].agg(['count'])
p

,count
overall,
1.0,181345
2.0,226940
3.0,499226
4.0,1111154
5.0,2980711


In [ ]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.20)
 
svd = SVD(n_epochs=10)
 
# Train the algorithm on the trainset
svd.fit(trainset)

The below cell shows the books liked by the particular reader, considering only 5 start ratings

In [ ]:
csv_merged[(csv_merged['reviewerID'] == "A1REUF3A1YCPHM") & (csv_merged["sentiment"] > 4)]

,Unnamed: 0,overall,reviewerID,asin,sentiment
0,0,5.0,A1REUF3A1YCPHM,0001713353,4.9396
4445789,4446340,5.0,A1REUF3A1YCPHM,0330293133,4.5386
4758108,4758694,5.0,A1REUF3A1YCPHM,0345522478,4.7216
4846901,4847505,3.0,A1REUF3A1YCPHM,034554854X,4.9888


The below cell shows the rating the reader might give to a book chosen at random from the dataset

In [ ]:
svd.predict("A1REUF3A1YCPHM", '0356500586').est

4.090749390973873

In [ ]:
def recommendation(model, user_id, data, n_items):
   
   allUniqueBooks = csv_merged["asin"].unique()   # Generate list of all books from the dataset
   print("Total unique books in the dataset:", len(allUniqueBooks))
   allUniqueBooks = pd.DataFrame(allUniqueBooks, columns=['Books']) 
   print(allUniqueBooks.head())
   booksRead = csv_merged.loc[csv_merged["reviewerID"] == user_id, "asin"]   # Get all books read by the reader
   print("Following books have read by the user:" + "\n", booksRead)
   booksNotRead = np.setdiff1d(allUniqueBooks, booksRead)     # Get all books not read by the reader
   print("List of books not read by the user:" + "\n", booksNotRead)
 
   booksPrediction = allUniqueBooks.copy()
   booksPrediction = booksPrediction.reset_index()
   booksPrediction['Recommendation_Score'] = booksPrediction['Books'].apply(lambda x: svd.predict(user_id, x).est)
   booksPrediction = booksPrediction.sort_values('Recommendation_Score', ascending=False)
   print(booksPrediction.head(n_items))
 
 
# define which user ID that we want to give recommendation
userID = 'A1REUF3A1YCPHM'
# define how many top-n movies that we want to recommend
n_items = 2
# generate recommendation using the model that we have trained
recommendation(svd,userID,data,n_items)

Total unique books in the dataset: 74405
        Books
0  0001713353
1  0001061240
2  0001932349
3  0001712799
4  0002006448
Following books have read by the user:
 0          0001713353
4445789    0330293133
4758108    0345522478
4846901    034554854X
Name: asin, dtype: object
List of books not read by the user:
 ['0000013765' '0000230022' '0000913154' ... '0373010176' '0373010192'
 '0373010222']
       index       Books  Recommendation_Score
0          0  0001713353              4.090749
49627  49627  0307455386              4.090749
